<h2>IMPORTS</h2>

In [27]:
# imports go here
import db

<h2>Data Retrieval</h2>

In [74]:
# dh = db.DatabaseHandler("testdb.db")  # db connection


def GetDF(dh:db.DatabaseHandler, selector: str, eventID: int, splitBySentences: bool = False):
    df = dh.get_recordDataJoinedDF(selector=selector, ID=eventID)
    if splitBySentences:
        # df.set_index('id', inplace=True)
        df['answer'] = df['answer'].str.split('.')
        df = df.explode("answer", True)
        df = df.drop(df[df["answer"] == ""].index)
    return df


# df = GetDF("event_id", 19, True)
